In [1]:
from pysgx import info, stocks, util, dm_test
import importlib
import pandas as pd
import numpy as np

c:\Users\vic52\Documents\GitHub\sgx3000\pysgx\dm_test.py:108: SyntaxWarning: invalid escape sequence '\d'
  comp = re_compile("^\d+?\.\d+?$")


In [91]:
# Re-run this cell if pysgx module have hot changes
importlib.reload(info);
importlib.reload(stocks);
importlib.reload(util);

# Portfolio

In [2]:
# Load ALL stocks!
all_stocks = stocks.loads(info.all_tickers)

def get_portfolio(tickers):
    portfolio = []
    for stock in all_stocks:
        if stock.ticker in tickers:
            portfolio.append(stock)
    return portfolio

# Rolling Test using AddBack Close Return

In [9]:
period_start = util.YearMonth(2000, 1)
period_last = util.YearMonth(2023, 1)

ByAddBackClose = []
ByAdjClose = []

while period_start <= period_last:
    returns = dict()
    for stock in all_stocks:
        period_end = util.get_next(period_start, 12)
        adtv = stock.get_adtv(period_start, period_end)

        if adtv > 100000:
            r = stocks.get_return(period_start, period_end, stock, False)
            if not np.isnan(r.ByAddBackClose):
                returns[stock.ticker] = r.ByAddBackClose

    # sort to find top 10 return stocks using AddBack Close
    returns2 = sorted(returns.items(), key=lambda x: x[1], reverse=True)[:10]
    top_returns = {k[0]: returns[k[0]] for k in returns2}

    print("Evaluate 1Y since: ", period_start, top_returns.keys())
    eval_period_end = util.get_next(period_end, 3)

    r = stocks.get_portfolio_return(period_end, eval_period_end, get_portfolio(top_returns.keys()))
    print("backtest 1Q", period_end, eval_period_end, r)
    ByAdjClose.append(r.ByAdjClose)
    ByAddBackClose.append(r.ByAddBackClose)
    print("")

    # roll to next quarter
    period_start = util.get_next(period_start, 3)

Evaluate 1Y since:  2000-01-01 dict_keys(['L02', '5AB', 'J36', 'CTO', 'H78', 'BTJ', 'BDA', 'LJ3', 'KUH', 'H02'])
backtest 1Q 2001-01-01 2001-04-01 ByAdjClose: -0.153297 ByAddBackClose: -0.152858

Evaluate 1Y since:  2000-04-01 dict_keys(['L02', 'LJ3', 'J36', 'H78', 'H02', 'C52', 'U14', '5AB', 'S63', 'S20'])
backtest 1Q 2001-04-01 2001-07-01 ByAdjClose:  0.046727 ByAddBackClose:  0.044415

Evaluate 1Y since:  2000-07-01 dict_keys(['600', 'T14', 'AZR', 'L02', 'LJ3', 'G07', 'H02', 'F13', 'WPC', 'J36'])
backtest 1Q 2001-07-01 2001-10-01 ByAdjClose: nan ByAddBackClose: -0.088402

Evaluate 1Y since:  2000-10-01 dict_keys(['AZR', 'T14', '600', 'D01', 'G07', 'LJ3', 'S20', 'H02', 'F13', 'BAZ'])
backtest 1Q 2001-10-01 2002-01-01 ByAdjClose:  0.244899 ByAddBackClose:  0.244330

Evaluate 1Y since:  2001-01-01 dict_keys(['T14', '600', 'AZR', 'G13', 'F13', 'D01', 'G07', 'BAZ', 'T15', 'G92'])
backtest 1Q 2002-01-01 2002-04-01 ByAdjClose:  0.089573 ByAddBackClose:  0.088380

Evaluate 1Y since:  2001-0

In [86]:
df = pd.DataFrame(ByAddBackClose, columns=['ByAddBackClose'])
returns = df['ByAddBackClose'] + 1
returns.product()

3.07166240833986

In [85]:
df = pd.DataFrame(ByAdjClose, columns=['ByAdjClose'])
returns = df['ByAdjClose'].dropna() + 1
returns.product()

6.466738845153519

# Rolling Test using Adj Close Return

In [21]:
period_start = util.YearMonth(2000, 1)
period_last = util.YearMonth(2023, 1)

ByAddBackClose = []
ByAdjClose = []

while period_start <= period_last:
    returns = dict()
    for stock in all_stocks:
        period_end = util.get_next(period_start, 12)
        adtv = stock.get_adtv(period_start, period_end)

        if adtv > 100000:
            r = stocks.get_return(period_start, period_end, stock, False)
            if not np.isnan(r.ByAdjClose):
                returns[stock.ticker] = r.ByAdjClose

    # sort to find top 10 return stocks using Adj Close
    returns2 = sorted(returns.items(), key=lambda x: x[1], reverse=True)[:10]
    top_returns = {k[0]: returns[k[0]] for k in returns2}

    print("Evaluate 1Y since: ", period_start, top_returns.keys())
    eval_period_end = util.get_next(period_end, 3)
    if len(top_returns) == 10:
        r = stocks.get_portfolio_return(period_end, eval_period_end, get_portfolio(top_returns.keys()))
        print("backtest 1Q", period_end, eval_period_end, r)
        ByAdjClose.append(r.ByAdjClose)
        ByAddBackClose.append(r.ByAddBackClose)
    print("")

    # roll to next quarter
    period_start = util.get_next(period_start, 3)

Evaluate 1Y since:  2000-01-01 dict_keys(['L02', '5AB', 'J36', 'H78', 'CTO', 'BTJ', 'LJ3', 'BDA', 'H02', 'KUH'])
backtest 1Q 2001-01-01 2001-04-01 ByAdjClose: -0.153297 ByAddBackClose: -0.152858

Evaluate 1Y since:  2000-04-01 dict_keys(['L02', 'LJ3', 'J36', 'H78', 'H02', 'C52', 'U14', '5AB', 'S63', 'S20'])
backtest 1Q 2001-04-01 2001-07-01 ByAdjClose:  0.046727 ByAddBackClose:  0.044415

Evaluate 1Y since:  2000-07-01 dict_keys(['600', 'AZR', 'L02', 'LJ3', 'G07', 'H02', 'F13', 'CTO', 'WPC', 'J36'])
backtest 1Q 2001-07-01 2001-10-01 ByAdjClose: -0.205341 ByAddBackClose: -0.211244

Evaluate 1Y since:  2000-10-01 dict_keys(['AZR', '600', 'D01', 'G07', 'LJ3', 'S20', 'H02', 'F13', 'BAZ', 'M01'])
backtest 1Q 2001-10-01 2002-01-01 ByAdjClose:  0.218573 ByAddBackClose:  0.218573

Evaluate 1Y since:  2001-01-01 dict_keys(['AZR', '600', 'G13', 'F13', 'D01', 'G07', 'BAZ', 'T15', 'G92', 'S41'])
backtest 1Q 2002-01-01 2002-04-01 ByAdjClose:  0.081422 ByAddBackClose:  0.081422

Evaluate 1Y since:  

In [22]:
df = pd.DataFrame(ByAddBackClose, columns=['ByAddBackClose'])
returns = df['ByAddBackClose'] + 1
returns.product()

1.8382072607663853

In [23]:
df = pd.DataFrame(ByAdjClose, columns=['ByAdjClose'])
returns = df['ByAdjClose'].dropna() + 1
returns.product()

98.17057979089792

# Manual portfolio

In [20]:
dy1 = ["M01", "5DD", "CC3", "i07", "BN4"]
dy2 = ["L38", "E5H", "1D0", "Z74", "CY6U"]
# ww = ["A04", "5VP", "BQD", "F03", "NO4", "41B", "A34", "T14", "Q0X"]

ww = ["ES3"]

ByAddBackClose = []
ByAdjClose = []


period_start = util.YearMonth(2008, 1)
period_last = util.YearMonth(2024, 1)

portfolio = stocks.loads(ww)
while period_start <= period_last:
    period_end = util.get_next(period_start, 3)

    # returns = stocks.get_returns(period_start, period_end, portfolio)
    # w1 = stocks.get_weights_by_returns(returns)

    r = stocks.get_portfolio_return(period_start, period_end, portfolio)
    print(period_start, period_end, r)
    ByAdjClose.append(r.ByAdjClose)
    ByAddBackClose.append(r.ByAddBackClose)

    # r = stocks.get_portfolio_return(period_end, backtest_period_end, portfolio, w1.ByAddBackClose)
    # print(backtest_period_start, backtest_period_end, r)

    # roll to next quarter
    period_start = util.get_next(period_start, 3)

stocks.get_overall_return(ByAdjClose, ByAddBackClose)

2008-01-01 2008-04-01 ByAdjClose: -0.144110 ByAddBackClose: -0.144110
2008-04-01 2008-07-01 ByAdjClose: -0.018033 ByAddBackClose: -0.018033
2008-07-01 2008-10-01 ByAdjClose: -0.194631 ByAddBackClose: -0.194631
2008-10-01 2009-01-01 ByAdjClose: -0.244813 ByAddBackClose: -0.244813
2009-01-01 2009-04-01 ByAdjClose: -0.085106 ByAddBackClose: -0.085106
2009-04-01 2009-07-01 ByAdjClose:  0.377907 ByAddBackClose:  0.377907
2009-07-01 2009-10-01 ByAdjClose:  0.142858 ByAddBackClose:  0.142857
2009-10-01 2010-01-01 ByAdjClose:  0.096654 ByAddBackClose:  0.096654
2010-01-01 2010-04-01 ByAdjClose: -0.010101 ByAddBackClose: -0.010101
2010-04-01 2010-07-01 ByAdjClose: -0.023649 ByAddBackClose: -0.023649
2010-07-01 2010-10-01 ByAdjClose:  0.097902 ByAddBackClose:  0.097902
2010-10-01 2011-01-01 ByAdjClose:  0.031446 ByAddBackClose:  0.031447
2011-01-01 2011-04-01 ByAdjClose: -0.054217 ByAddBackClose: -0.054217
2011-04-01 2011-07-01 ByAdjClose:  0.006309 ByAddBackClose:  0.006309
2011-07-01 2011-10-0

In [8]:
# stock = stocks.load("D05")
# print(stock.get_sector())
# df = util.to_df(info.get_all_avg_vol(), ['Ticker', 'AvergeVolume'])
# df.AvergeVolume = df.AvergeVolume / 10000
# df = df[df.AvergeVolume>1].sort_values(by=['AvergeVolume'])

# df = util.to_df(info.get_all_mc(), ['Ticker', 'MarketCap'])
# df.MarketCap = df.MarketCap / 1000000
# df = df[df.MarketCap>1000].sort_values(by=['MarketCap'])

# items = stocks.loads(df.Ticker.to_list())

period_start = util.YearMonth(2000, 1)
period_last = util.YearMonth(2024, 1)

y_yields = []
while period_start < period_last:
    period_end = util.get_next(period_start, 12)
    count = 0
    total_yield = 0
    for stock in all_stocks:
        df = stock.dividend
        dividend = df.loc[df["Day"] > str(period_start)].loc[df["Day"] < str(period_end)].Dividends.sum()
        sell_index = stock.price['Day'].searchsorted(str(period_end))
        if dividend > 0:
            dyield = dividend / stock.price.iloc[sell_index].Close
            count += 1
            total_yield += dyield
    y_yield = total_yield / count
    y_yields.append(y_yield)
    print(period_start, ": ", count, " ", y_yield)
    period_start = period_end

print(sum(y_yields) / len(y_yields))

2000-01-01 :  109   0.047040682895506294
2001-01-01 :  124   0.06357748477897936
2002-01-01 :  114   0.05655976607323115
2003-01-01 :  132   0.05138115624439527
2004-01-01 :  145   0.051459969970127625
2005-01-01 :  172   0.0724260613838125
2006-01-01 :  202   0.061370292063319586
2007-01-01 :  225   0.05993048506935729
2008-01-01 :  249   0.11842908787730255
2009-01-01 :  225   0.061051344779131975
2010-01-01 :  253   0.06526285336138526
2011-01-01 :  287   0.09221663132394683
2012-01-01 :  288   0.06942911037241112
2013-01-01 :  308   0.06373178925316576
2014-01-01 :  320   0.09257821969571672
2015-01-01 :  304   0.06790733199557104
2016-01-01 :  294   0.08978533747252226
2017-01-01 :  297   0.06304922512826906
2018-01-01 :  321   0.07262295335139904
2019-01-01 :  307   0.07756952077879974
2020-01-01 :  303   0.08269244711683504
2021-01-01 :  308   0.13286366346367773
2022-01-01 :  197   0.10787902286600858
2023-01-01 :  172   0.06832384807589578
0.07454742855794864


# DM test

In [ ]:
period_start = util.YearMonth(2000, 1)
period_last = util.YearMonth(2023, 1)

ByAddBackClose = []
ByAddBackCloseW1 = []
ByAddBackCloseW2 = []

while period_start <= period_last:
    returns = dict()
    stock_returns = dict()
    for stock in all_stocks:
        period_end = util.get_next(period_start, 12)
        adtv = stock.get_adtv(period_start, period_end)

        if adtv > 100000:
            r = stocks.get_return(period_start, period_end, stock, False)
            returns[stock.ticker] = r.ByAddBackClose
            stock_returns[stock.ticker] = r

    # sort to find top 10 return stocks using AddBack Close
    returns2 = sorted(returns.items(), key=lambda x: x[1], reverse=True)[:10]
    top_returns = {k[0]: returns[k[0]] for k in returns2}

    portfolio = get_portfolio(top_returns.keys())

    top_stock_returns = [stock_returns[k[0]] for k in returns2]
    w1 = stocks.get_weights_by_returns(top_stock_returns).ByAddBackClose

    risks = stocks.get_risks(period_start, period_end, portfolio)
    w2 = stocks.get_weights_by_risks(risks).ByAddBackClose

    eval_period_end = util.get_next(period_end, 3)

    r = stocks.get_portfolio_return(period_end, eval_period_end, portfolio)
    ByAddBackClose.append(r.ByAddBackClose)

    r = stocks.get_portfolio_return(period_end, eval_period_end, portfolio, w1)
    ByAddBackCloseW1.append(r.ByAddBackClose)

    r = stocks.get_portfolio_return(period_end, eval_period_end, portfolio, w2)
    ByAddBackCloseW2.append(r.ByAddBackClose)

    # roll to next quarter
    period_start = util.get_next(period_start, 3)

print("ByAddBackClose")
print(ByAddBackClose)
print("ByAddBackCloseW1")
print(ByAddBackCloseW1)
print("ByAddBackCloseW2")
print(ByAddBackCloseW2)

In [12]:
# ByAddBackClose
s0_return = [-0.15285830881861276, 0.04441541329149104, -0.088402378610857, 0.24433046442410372, 0.0883799568020283, -0.08489086962964883, -0.16495054637832032, 0.05461221209610552, 0.10200050295096036, 0.2338967515183634, 0.18120520920749306, 0.025300184911485023, 0.15144374764251517, -0.18644310861424324, -0.006493237185101017, -0.03836095574976314, -0.08239579469669979, 0.08610782868046507, 0.11664684439916914, -0.013695876059588682, 0.23701916490905584, -0.12349164865016672, 0.08732470660854186, -0.011155704216471413, -0.05475815698943616, 0.4796924198084972, 0.1029746285974737, -0.22588015514154908, -0.28046509354705007, -0.051580339465586364, -0.3567243641357939, -0.2306828169568687, -0.11973713095288765, 0.2109021825521924, 0.4016009682615337, -0.02218195502185547, 0.0221567204780441, -0.11757836721760974, 0.06970326374661977, -0.036783208461998965, -0.04164288062252413, -0.016848634932226422, -0.03875988995141196, -0.07457857322355779, 0.27424000199288096, 0.09199052244206304, 0.3307733776873145, 0.28056604593993323, 0.35595224722587404, 0.08031125674739543, 0.20900121090478796, -0.20039421888243175, -0.1422219898994231, -0.16938626362517656, -0.03174688822144675, -0.05438667913226916, 0.013878754882218656, 0.11402170357270339, -0.12513047802493457, 0.0872114720379353, -0.13230395354194904, 0.19035747287035032, 0.09255175479983577, 0.05376730208926507, 0.22068577307800927, -0.11175077081145504, -0.049349282989739826, 0.008102616979111636, 0.01783886672294005, -0.18438575356323247, -0.06830966432367953, -0.20310356465071072, 0.15735285245964414, -0.006026759120876234, -0.01543149018723396, 0.1485107681807331, -0.46558066446896035, 0.974639716023122, 0.17003365120758354, -0.02150993469673424, 0.19915741106802914, 0.11427547242454539, 0.14449503215807866, -0.028351514834700164, 0.38135545959548045, -0.17801574108833082, -0.1749560063419107, 0.03341264980943798, 0.044521647776395286, 0.01682785407000275, -0.006248194112466563, -0.03319231733179102, -0.024978058449474712]

# ByAddBackCloseW1
s1_return = [-0.20001710091756622, 0.15540044210511766, -0.11859881817291108, 0.2983122450431943, 0.0868398759348324, -0.12542446049381817, -0.20220785505011504, 0.05865790679072406, 0.07303617583620824, 0.25095202438538666, 0.08969616935525125, 0.03176637735091256, 0.15827350990534297, -0.26653892793435113, -0.015259919249663193, 0.003594821275029936, -0.05123504252591182, 0.13826403223924483, 0.08364805230677216, -0.034119933293853155, 0.2410083610697461, -0.12638638326814874, 0.1409391659833895, -0.09373093535317574, -0.04626455309703413, 0.3503282264693986, 0.2953428754847402, -0.24057733466209769, -0.26246760365916677, -0.22361506827092004, -0.4240740125044331, -0.35526575822978873, 0.5797188813968784, 0.41403721669769555, 0.3639981154339941, -0.012697553481340803, -0.0023563100736405454, -0.11690351481975776, 0.0755184198725493, -0.02990513153499819, -0.024465771705704138, -0.02174309807398537, -0.0605608006783991, -0.04075832620086401, 0.21820179477030552, 0.07288420152599133, 0.3148868460265421, 0.16828544273750867, 0.31666841992929035, -0.05156604093165883, 0.2188227219223031, -0.07134360921705656, -0.14044921962290924, -0.2039908961064354, -0.0340556069150546, -0.0985802167352831, 0.04836023043331749, 0.1153198331044662, -0.05405066954485042, 0.19906825161047031, 0.07687186380765906, 0.17470418230334978, 0.12164908684641879, 0.09396470630774137, 0.17284780404395567, -0.14586908855153868, -0.057640281364134426, -0.028381734731711628, 0.03090181682741609, -0.22073544901946787, -0.12849882477472982, -0.24867594803924173, 0.056525692632758764, -0.025400703360895872, -0.026495178537536516, 0.1519268004204091, -0.45907521668029927, 1.0012853955684005, 0.31754331455994894, -0.1037623263494014, 0.1786020908239908, 0.12838919940558224, 0.015757300064673407, -0.06717050620438919, 0.37015379595195563, -0.16107323149738897, -0.2307417961015681, 0.008981980645851425, 0.17735089388281908, -0.1195975821737507, -0.00011080089053170164, -0.0384871974398854, -0.016625518448852444]

# ByAddBackCloseW2
s2_return = [-0.14684056285824468, 0.03308163231304791, -0.1090086998871648, 0.23751631497902526, 0.08965787628341876, -0.08012647258370167, -0.17355461489229929, 0.05125241696534283, 0.10635143529548145, 0.23366672021009166, 0.1945471852270691, 0.023675727828367486, 0.16290823451831035, -0.17379581741106426, -0.005140852800104716, -0.03651410146198267, -0.08553086437022477, 0.08914919912426286, 0.11369106422869897, -0.014280534651125092, 0.23693353723235738, -0.12427120125379751, 0.02408108194514539, -0.01515566418758476, -0.04364123242026778, 0.5058745982501307, 0.0801549519182231, -0.24451674561638048, -0.28444592106796956, -0.0445663711945195, -0.353609826786269, -0.22883705523872894, -0.11726519942613557, 0.2208101676940229, 0.4103434268928471, -0.015891178191261984, 0.04528061027423274, -0.11688682120325249, 0.06897258906077958, -0.03155330219665291, -0.04062219291064524, -0.009011575261779037, -0.0370630067509137, -0.0775990410410669, 0.2794943788027251, 0.11745199190241144, 0.3119884227791262, 0.2959567808179755, 0.35346104703871184, 0.0995688932754597, 0.21098082118421907, -0.19827606641218912, -0.15329259528352143, -0.16046295895839446, -0.02854270448442676, -0.044040247877653836, 0.019452478551008144, 0.11521132419116818, -0.1341423017556126, 0.0839262605581221, -0.14308394786842613, 0.228580920966294, 0.10215771646827723, 0.04926174680028303, 0.2205413851761361, -0.1089231677545609, -0.045940137005316684, 0.012423153972983974, 0.04502664684470052, -0.18908688390275558, -0.07804459694946884, -0.19297291388516144, 0.14997258554811613, 0.000548517141804131, -0.01622681645797771, 0.15156464928115365, -0.45359364380152206, 1.0268240951961263, 0.20547508118509958, -0.024370644627460143, 0.18872300128793884, 0.14389557764444538, 0.13419123939856054, -0.027026499455437766, 0.32489087612900946, -0.17374387492856866, -0.17458701986548764, 0.036951959632490324, 0.05875252001063429, 0.03563401038049434, 0.00010051507306975058, -0.04837750519881171, -0.02036274261340815]

print("s0 vs s1")
rt = dm_test.test_strategy(s0_return, s1_return)
print(rt)

print("s0 vs s2")
rt = dm_test.test_strategy(s0_return, s2_return)
print(rt)

print("s1 vs s2")
rt = dm_test.test_strategy(s1_return, s2_return)
print(rt)

s0 vs s1
dm_return(DM=0.11823563676620152, p_value=0.9061387226118884)
s0 vs s2
dm_return(DM=1.4118301667098272, p_value=0.16137333663157002)
s1 vs s2
dm_return(DM=0.0953629138384972, p_value=0.9242339807716745)


In [13]:
rt = dm_test.dm_test(s0_return, s1_return, s2_return)
print(rt)

dm_return(DM=1.8331375977777478, p_value=0.07001611485180186)


In [4]:
s1 = pd.Series(s0_return)
s2 = pd.Series(s1_return)
s3 = pd.Series(s2_return)

print("overall return")
print((s1+1).product() - 1, "\t", (s2+1).product()-1, "\t", (s3+1).product()-1)

print("mean")
print(s1.mean(), "\t", s2.mean(), "\t", s3.mean())

print("risk")
print(s1.var(), "\t", s2.var(), "\t", s3.var())

overall return
2.07166240833986 	 1.384823130491502 	 2.657403780155815
mean
0.029004794431215623 	 0.03023370650106794 	 0.03124872304010862
risk
0.03749130452568611 	 0.045624301425866334 	 0.038815941108419884


# Divident Stats

In [4]:
stock_with_dividend = 0
for s in all_stocks:
    if len(s.dividend) > 0:
        stock_with_dividend +=1

print(len(all_stocks), " ", stock_with_dividend)

period_start = util.YearMonth(2015, 1)
period_last = util.YearMonth(2023, 1)

portfolio = stocks.loads(ww)
while period_start <= period_last:
    period_end = util.get_next(period_start, 12)

708   542


# Adj Close return vs AddBack Return

In [12]:
stock = stocks.load("C38U")
stocks.list_dividend("2022-03-03", stock)
r = stocks.get_return("2022-03-03", "2024-03-16", stock)
print(r)

== csv

Day,Close,Original Close,Dividends,Adjustment Factor
2024-02-14,1.91,1.91,0.0545,0.9722575719012472
2023-08-08,1.886869,1.9407089793193715,0.053,0.9734163809513997
2023-02-08,1.922083,2.030916897832926,0.0536,0.9742866080597751
2022-08-04,2.020652,2.191415895484169,0.0522,0.9767339854807299


total_adjustment_factor: 0.900622964950044

# Return by Adj Close BUY: 2022-03-03 SELL: 2024-03-15
buy: 1.918871 sell: 1.920000 gain: 0.001129

return: 0.06% (0.000588)
Total Dividend: 0.213300

# Actual Return by adding back dividend
buy: 2.130000 sell: 2.133300 gain: 0.003300
return: 0.15% (0.001549)

# Return difference
-62.02% (-0.620236)
ByAdjClose:  0.000588 ByAddBackClose:  0.001549


In [183]:
# stock = stocks.load("D05")
stock = stocks.load("D8DU")
df = stocks.get_addback_close(stock, "2022-01-01", "2024-01-01")

return_adj_close = df["Adj Close"].pct_change().dropna()
return_addback_close = df["AddBackClose"].pct_change().dropna()

print(return_addback_close.mean())
print(return_adj_close.mean())
return_adj_close.std()  / return_addback_close.std() - 1
# return_adj_close.mean()  / return_addback_close.mean() - 1

-0.0002463114195503293
-0.00044110314428133615


0.04505055363062538

In [195]:
df = pd.concat([return_addback_close, return_adj_close], axis=1)
df.cov().iloc[0, 1]

0.000718800466679042

# Stock Market Cap Distribution

In [ ]:
df = util.to_df(info.get_all_mc(), ['Ticker', 'MarketCap'])
df.MarketCap = df.MarketCap / 1000000
plt = df[df.MarketCap<100].MarketCap.plot.hist(bins=50, alpha=0.5, title='SGX Stock Market Cap Distribution - Below(100M SGD)')
plt.legend(['Total 365 stocks'])
plt.set_xlabel("Market Cap (1M)")
plt.get_figure().savefig('mc_100M.png')

plt = df[df.MarketCap>100][df.MarketCap<1500].MarketCap.plot.hist(bins=50, alpha=0.5, title='SGX Stock Market Cap Distribution - 100M~1.5B SGD)')
plt.legend(['Total 188 stocks'])
plt.set_xlabel("Market Cap (1M)")
plt.get_figure().savefig('mc_100M_1.5B.png')

plt = df[df.MarketCap>1500][df.MarketCap<30000].MarketCap.plot.hist(bins=50, alpha=0.5, title='SGX Stock Market Cap Distribution - 1.5B~30B SGD)')
plt.legend(['Total 62 stocks'])
plt.set_xlabel("Market Cap (1M)")
plt.get_figure().savefig('mc_1.5B_20B.png')

df[df.MarketCap>10000].sort_values(by=['MarketCap']).to_csv("mc_10B_plus.csv", index=False, float_format='%.6f')

# Average Daily Trading Volume Hist

In [ ]:
df = util.to_df(info.get_all_avg_vol(), ['Ticker', 'AvergeVolume'])
df.AvergeVolume = df.AvergeVolume / 10000

plt = df[df.AvergeVolume<=1].plot.hist(bins=50, alpha=0.5, title='SGX Average Daily Trading Volume (ADTV) - Below(10K SGD)')
plt.set_xlabel("ADTV (10K)")
plt.legend(['ADTV Total 365 stocks'])
plt.get_figure().savefig('adtv_10K.png')

plt = df[1<df.AvergeVolume][df.AvergeVolume<=30].plot.hist(bins=50, alpha=0.5, title='SGX Average Daily Trading Volume (ADTV) 10K~300K SGD')
plt.set_xlabel("ADTV (10K)")
plt.legend(['ADTV Total 207 stocks'])
plt.get_figure().savefig('adtv_10K_300K.png')

plt = df[df.AvergeVolume>30][df.AvergeVolume<=300].plot.hist(bins=50, alpha=0.5, title='SGX Average Daily Trading Volume (ADTV) 300K~3M SGD')
plt.set_xlabel("ADTV (10K)")
plt.legend(['ADTV Total 68 stocks'])
plt.get_figure().savefig('adtv_300K_3M.png')

df[df.AvergeVolume>300].sort_values(by=['AvergeVolume']).to_csv("adtv_3M_plus.csv", index=False, float_format='%.6f')

# STI Estimate

In [ ]:
es3 = stocks.load("ES3")
df = stocks.get_addback_close(es3, "2010-03-01", "2024-03-15")
df["Adj Close"] = df["Adj Close"] * 990
df["AddBackClose"] = df["AddBackClose"] * 1010
df["Close"] = df["Close"] * 1000

plt = df.plot(x="Day", y = ["AddBackClose", "Close", "Adj Close"], linewidth=0.8, figsize=(15, 10), title="Esitmated STI")
plt.legend(['Estimated STI if add back dividend', 'STI', 'Estimated STI if use Adj Close'])
plt.get_figure().savefig('STI_estimated.png')

# Strategy comparison

In [18]:
# Load ALL stocks!
all_stocks = stocks.loads(info.all_tickers)

def get_portfolio(tickers):
    portfolio = []
    for stock in all_stocks:
        if stock.ticker in tickers:
            portfolio.append(stock)
    return portfolio

## High market cap high P/E ratio

In [82]:
# Investor 3-1 portfolio return
dy1 = ["M01", "5DD", "CC3", "i07", "BN4"]
dy2 = ["L38", "E5H", "1D0", "Z74", "CY6U"]
#ww = ["A04", "5VP", "BQD", "F03", "NO4", "41B", "A34", "T14", "Q0X"]


ByAddBackClose = []
ByAdjClose = []


period_start = util.YearMonth(2022, 1)
period_last = util.YearMonth(2022, 12)

portfolio = stocks.loads(dy1)
while period_start <= period_last:
    period_end = util.get_next(period_start, 3)

    #returns = stocks.get_returns(period_start, period_end, portfolio)
    #w1 = stocks.get_weights_by_returns(returns)

    r = stocks.get_portfolio_return(period_start, period_end, portfolio)
    print(period_start, period_end, r)
    ByAdjClose.append(r.ByAdjClose)
    ByAddBackClose.append(r.ByAddBackClose)

    #r = stocks.get_portfolio_return(period_end, backtest_period_end, portfolio, w1.ByAddBackClose)
    #print(backtest_period_start, backtest_period_end, r)

    # roll to next quarter
    period_start = util.get_next(period_start, 3)

stocks.get_overall_return(ByAdjClose, ByAddBackClose)

2022-01-01 2022-04-01 ByAdjClose:  0.003161 ByAddBackClose:  0.003248
2022-04-01 2022-07-01 ByAdjClose: -0.055573 ByAddBackClose: -0.055683
2022-07-01 2022-10-01 ByAdjClose: -0.033883 ByAddBackClose: -0.033613
2022-10-01 2023-01-01 ByAdjClose: -0.024424 ByAddBackClose: -0.024374
0.8929557795085016
0.8932239048359752


In [83]:
print(ByAddBackClose)

[0.0032478125859561474, -0.055683145690671974, -0.033612868611718354, -0.024374289777115657]


In [36]:
v1_r=ByAddBackClose
v1_r

[0.0271470647228335,
 0.12526051034443852,
 0.05214539230908373,
 -0.054634366212942,
 -0.0787152977163386,
 0.036515271642701075,
 -0.08358628253989185,
 0.015731283858857782,
 0.016887575636170982,
 -0.075590173767214,
 0.08767017128606211,
 -0.22989934788661281,
 0.18590482210337703,
 0.14361192398490602,
 0.12050465802104321,
 0.10810341256044242,
 0.029927346317255868,
 -0.02888471366718135,
 0.06577817661112005,
 0.0032478125859561474,
 -0.055683145690671974,
 -0.033612868611718354,
 -0.024374289777115657,
 0.001022908923912512]

In [5]:
print(ByAddBackClose)

[nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 0.11098336339032261, 0.017597321217329508, -0.017565624422355607, -0.07133982726848812, 0.07159418579449064, -0.1715003015370804, -0.2573099547384745, -0.25919147647279245, 0.46192578037754867, 0.18538514606394396, 0.036834674945772305, 0.02693945700310827, -0.022908989540734635, 0.21456932835717685, 0.04007752042936212, 0.0012437325915352956, 0.11662598158888327, -0.134665152679311, 0.06839210695222704, 0.07095039652038078, 0.013672165845963133, 0.05495594295853318, 0.00943975090754312, 0.7758047612918498, 0.26503580604819127, 0.024364601459183868, -0.031733782385490614, -0.02324210959602462, 0.03372771344681069, -0.057628368438236516, -0.05200634801064341, 0.010738735360127035, 0.03640557596946681, -0.10755872323485485, 0.06098091830050532, -0.009094908333034037, 0.033594857394246364, -0.004609469599860061, 0.025008966583831362, 0.113270002

## High market cap low P/E ratio

In [77]:
# Investor 3-2 portfolio return
dy1 = ["M01", "5DD", "CC3", "i07", "BN4"]
dy2 = ["L38", "E5H", "1D0", "Z74", "CY6U"]
# ww = ["A04", "5VP", "BQD", "F03", "NO4", "41B", "A34", "T14", "Q0X"]


ByAddBackClose = []
ByAdjClose = []


period_start = util.YearMonth(2017, 1)
period_last = util.YearMonth(2017, 12)

portfolio = stocks.loads(dy2)
while period_start <= period_last:
    period_end = util.get_next(period_start, 3)

    # returns = stocks.get_returns(period_start, period_end, portfolio)
    # w1 = stocks.get_weights_by_returns(returns)

    r = stocks.get_portfolio_return(period_start, period_end, portfolio)
    print(period_start, period_end, r)
    ByAdjClose.append(r.ByAdjClose)
    ByAddBackClose.append(r.ByAddBackClose)

    # r = stocks.get_portfolio_return(period_end, backtest_period_end, portfolio, w1.ByAddBackClose)
    # print(backtest_period_start, backtest_period_end, r)

    # roll to next quarter
    period_start = util.get_next(period_start, 3)

stocks.get_overall_return(ByAdjClose, ByAddBackClose)

2017-01-01 2017-04-01 ByAdjClose:  0.010481 ByAddBackClose:  0.010481
2017-04-01 2017-07-01 ByAdjClose: -0.054967 ByAddBackClose: -0.054930
2017-07-01 2017-10-01 ByAdjClose: -0.001289 ByAddBackClose: -0.001029
2017-10-01 2018-01-01 ByAdjClose:  0.052924 ByAddBackClose:  0.052984
1.0041814059282395
1.004539823147689


In [69]:
print(ByAddBackClose)

[0.0393129640712212, -0.05768356619331527, -0.008870449196184427, -0.009239495355153806]


# return outcome

In [33]:
period_start = util.YearMonth(2017, 4)
period_last = util.YearMonth(2023, 1)

ByAddBackClose = []
ByAddBackCloseW1 = []
ByAddBackCloseW2 = []

while period_start <= period_last:
    returns = dict()
    stock_returns = dict()
    for stock in all_stocks:
        period_end = util.get_next(period_start, 12)
        adtv = stock.get_adtv(period_start, period_end)

        if adtv > 100000:
            r = stocks.get_return(period_start, period_end, stock, False)
            returns[stock.ticker] = r.ByAddBackClose
            stock_returns[stock.ticker] = r

    # sort to find top 10 return stocks using AddBack Close
    returns2 = sorted(returns.items(), key=lambda x: x[1], reverse=True)[:10]
    top_returns = {k[0]: returns[k[0]] for k in returns2}

    portfolio = get_portfolio(top_returns.keys())

    top_stock_returns = [stock_returns[k[0]] for k in returns2]
    w1 = stocks.get_weights_by_returns(top_stock_returns).ByAddBackClose

    risks = stocks.get_risks(period_start, period_end, portfolio)
    w2 = stocks.get_weights_by_risks(risks).ByAddBackClose

    eval_period_end = util.get_next(period_end, 3)

    r = stocks.get_portfolio_return(period_end, eval_period_end, portfolio)
    ByAddBackClose.append(r.ByAddBackClose)

    r = stocks.get_portfolio_return(period_end, eval_period_end, portfolio, w1)
    ByAddBackCloseW1.append(r.ByAddBackClose)

    r = stocks.get_portfolio_return(period_end, eval_period_end, portfolio, w2)
    ByAddBackCloseW2.append(r.ByAddBackClose)

    # roll to next quarter
    period_start = util.get_next(period_start, 3)

print("ByAddBackClose")
print(ByAddBackClose)
print("ByAddBackCloseW1")
print(ByAddBackCloseW1)
print("ByAddBackCloseW2")
print(ByAddBackCloseW2)

ByAddBackClose
[-0.18438575356323247, -0.06830966432367953, -0.20310356465071072, 0.15735285245964414, -0.006026759120876234, -0.01543149018723396, 0.1485107681807331, -0.46558066446896035, 0.974639716023122, 0.17003365120758354, -0.02150993469673424, 0.19915741106802914, 0.11427547242454539, 0.14449503215807866, -0.028351514834700164, 0.38135545959548045, -0.17801574108833082, -0.1749560063419107, 0.03341264980943798, 0.044521647776395286, 0.01682785407000275, -0.006248194112466563, -0.03319231733179102, -0.024978058449474712]
ByAddBackCloseW1
[-0.22073544901946787, -0.12849882477472982, -0.24867594803924173, 0.056525692632758764, -0.025400703360895872, -0.026495178537536516, 0.1519268004204091, -0.45907521668029927, 1.0012853955684005, 0.31754331455994894, -0.1037623263494014, 0.1786020908239908, 0.12838919940558224, 0.015757300064673407, -0.06717050620438919, 0.37015379595195563, -0.16107323149738897, -0.2307417961015681, 0.008981980645851425, 0.17735089388281908, -0.119597582173750

In [34]:
s0_r=ByAddBackClose
s0_r

[-0.18438575356323247,
 -0.06830966432367953,
 -0.20310356465071072,
 0.15735285245964414,
 -0.006026759120876234,
 -0.01543149018723396,
 0.1485107681807331,
 -0.46558066446896035,
 0.974639716023122,
 0.17003365120758354,
 -0.02150993469673424,
 0.19915741106802914,
 0.11427547242454539,
 0.14449503215807866,
 -0.028351514834700164,
 0.38135545959548045,
 -0.17801574108833082,
 -0.1749560063419107,
 0.03341264980943798,
 0.044521647776395286,
 0.01682785407000275,
 -0.006248194112466563,
 -0.03319231733179102,
 -0.024978058449474712]

## 2017

In [78]:
# ByAddBackClose
s0_return = [0.01783886672294005, -0.18438575356323247, -0.06830966432367953, -0.20310356465071072]

# ByAddBackCloseW1
s1_return = [0.03090181682741609, -0.22073544901946787, -0.12849882477472982, -0.24867594803924173]

# ByAddBackCloseW2
s2_return = [0.04502664684470052, -0.18908688390275558, -0.07804459694946884, -0.19297291388516144]

# ByAddBackClosev1
v1_return = [0.11327000238812154, 0.0271470647228335, 0.12526051034443852, 0.05214539230908373]
v2_return = [0.010480931601673672, -0.05492964145838924, -0.0010286609881583492, 0.052984226028955875]

print("s0 vs v1")
rt = dm_test.test_strategy(s0_return, v2_return)
print(rt)

print("s1 vs v1")
rt = dm_test.test_strategy(s1_return, v2_return)
print(rt)

print("s2 vs v1")
rt = dm_test.test_strategy(s2_return, v2_return)
print(rt)

print("v1 vs v2")
rt = dm_test.test_strategy(v1_return, v2_return)
print(rt)



s0 vs v1
{'mean_difference': -0.11136674274969116, 'dm_test': -1.9972413840570438, 'p_value': 0.13969902280558016}
s1 vs v1
{'mean_difference': -0.14362881504752634, 'dm_test': -2.1685918737418204, 'p_value': 0.11861298027626105}
s2 vs v1
{'mean_difference': -0.10564615076919183, 'dm_test': -1.8079763510590785, 'p_value': 0.16833087214931966}
v1 vs v2
{'mean_difference': 0.07757902864509883, 'dm_test': 2.805209645966729, 'p_value': 0.06756036829246757}


## 2022

In [84]:

s0_return = [0.044521647776395286, 0.01682785407000275, -0.006248194112466563, -0.03319231733179102]

s1_return = [0.17735089388281908, -0.1195975821737507, -0.00011080089053170164, -0.0384871974398854]

s2_return = [0.05875252001063429, 0.03563401038049434, 0.00010051507306975058, -0.04837750519881171]

v1_return = [0.0032478125859561474, -0.055683145690671974, -0.033612868611718354, -0.024374289777115657]
v2_return = [0.0393129640712212, -0.05768356619331527, -0.008870449196184427, -0.009239495355153806]

print("s0 vs v1")
rt = dm_test.test_strategy(s0_return, v2_return)
print(rt)

print("s1 vs v1")
rt = dm_test.test_strategy(s1_return, v2_return)
print(rt)

print("s2 vs v1")
rt = dm_test.test_strategy(s2_return, v2_return)
print(rt)

print("v1 vs v2")
rt = dm_test.test_strategy(v1_return, v2_return)
print(rt)


s0 vs v1
{'mean_difference': 0.01459738426889319, 'dm_test': 0.6941068458363885, 'p_value': 0.5375355862979212}
s1 vs v1
{'mean_difference': 0.013908965013020892, 'dm_test': 0.3173849734129288, 'p_value': 0.7717584269150695}
s2 vs v1
{'mean_difference': 0.02064752173470474, 'dm_test': 0.7542322586201686, 'p_value': 0.5055122627100315}
v1 vs v2
{'mean_difference': -0.018485486205029385, 'dm_test': -2.2941761003381966, 'p_value': 0.10555083324484736}


In [65]:
import pandas as pd

gdp = pd.read_csv("REITs analysis/SGgdp.csv")

# Access data by column names or index
print(gdp.head())  # View the first few rows


   Quarter  GDP
0  2017 Q4  6.4
1  2018 Q1  6.7
2  2018 Q2  8.1
3  2018 Q3  5.0
4  2018 Q4  8.4


In [70]:
import numpy as np
from numpy import nan
import statsmodels.api as sm

# Calculate the difference between s0_r and v1_r
difference = np.subtract(s0_r, v1_r)

# Check if gdp is a NumPy array
if not isinstance(gdp, np.ndarray):
    gdp = gdp.to_numpy()  # Convert to NumPy array if necessary

# Perform the regression using OLS
try:
    model = sm.OLS(difference, gdp).fit()
    print(model.summary())
except Exception as e:
    print("Error:", e)  # Handle potential errors


Error: ufunc 'isfinite' not supported for the input types, and the inputs could not be safely coerced to any supported types according to the casting rule ''safe''


In [85]:
# Data as a dictionary
data = {
    's0 vs v2': {'mean_difference': 0.01459738426889319, 'dm_test': 0.6941068458363885, 'p_value': 0.5375355862979212},
    's1 vs v2': {'mean_difference': 0.013908965013020892, 'dm_test': 0.3173849734129288, 'p_value': 0.7717584269150695},
    's2 vs v2': {'mean_difference': 0.02064752173470474, 'dm_test': 0.7542322586201686, 'p_value': 0.5055122627100315},
    'v1 vs v2': {'mean_difference': -0.018485486205029385, 'dm_test': -2.2941761003381966, 'p_value': 0.10555083324484736},
}

# Extract keys (comparison pairs) and values (statistics) into separate lists
comparison_pairs = list(data.keys())
statistics = list(data.values())

# Create a DataFrame with columns for comparison pairs, mean difference, dm_test, and p_value
df = pd.DataFrame(statistics, columns=['mean_difference', 'dm_test', 'p_value'], index=comparison_pairs)

# Print the DataFrame as a table
print(df.to_string())

          mean_difference   dm_test   p_value
s0 vs v2         0.014597  0.694107  0.537536
s1 vs v2         0.013909  0.317385  0.771758
s2 vs v2         0.020648  0.754232  0.505512
v1 vs v2        -0.018485 -2.294176  0.105551
